In [4]:
# API KEYS
from dotenv import dotenv_values
env = dotenv_values('.env') 

In [5]:
# Various Imports:
import json
import requests

import numpy as np
import pandas as pd
import networkx as nx
import osmnx as ox

# Data Preprocessing



### Preliminary Data Types
1. Weather & Sunset
2. Aesthetic
4. Safety
5. Infrastructure

In [10]:
# Get cities to make predictions for (0 / 4)
with open("cities.json") as json_file:
  cities = json.load(json_file)
  print(cities)

{'New_York': {'state': 'NY', 'country': 'USA', 'coords': [40.71427, -74.00597], 'timezone': 'New_York'}, 'Los_Angeles': {'state': 'CA', 'country': 'USA', 'coords': [34.0522, 118.2437], 'timezone': 'Los_Angeles'}, 'Chicago': {'state': 'IL', 'country': 'USA', 'coords': [41.8781, 87.6298], 'timezone': 'Chicago'}, 'Houston': {'state': 'TX', 'country': 'USA', 'coords': [29.7604, 95.3698], 'timezone': 'Denver'}, 'Pheonix': {'state': 'AZ', 'country': 'USA', 'coords': [33.4484, 112.074], 'timezone': 'Denver'}, 'Philadelphia': {'state': 'PA', 'country': 'USA', 'coords': [39.9526, 75.1652], 'timezone': 'New_York'}}


In [7]:
# Weather & Sunset Data (1 / 4)

# Source: Open Meteo
# Reference: https://open-meteo.com/en/docs
# Reliability: 10

cities = ["New_York", "Los_Angeles", "Chicago", "Houston", "Pheonix", "Philadelphia"]
coords = [[40.71427, -74.00597], [34.0522, 118.2437], [41.8781, 87.6298], [29.7604, 95.3698], [33.4484, 112.0740], [39.9526, 75.1652]]
timezone = ["New_York", "Los_Angeles", "Chicago", "Denver", "Denver", "New_York"]

weather_data = []
for i, city in enumerate(cities):
  URL = f"https://api.open-meteo.com/v1/forecast?latitude={coords[i][0]}&longitude={coords[i][1]}&daily=sunset&timezone=America%2F{timezone[i]}&past_days=14&hourly=temperature_2m,relativehumidity_2m,windspeed_10m,rain&temperature_unit=fahrenheit"
  response = requests.get(url = URL).json()

  if response:
    # Weather
    temp_avg = round(sum(response["hourly"]["temperature_2m"]) / len(response["hourly"]["temperature_2m"]), 2)
    humidity_avg = round(sum(response["hourly"]["relativehumidity_2m"]) / len(response["hourly"]["relativehumidity_2m"]), 2)
    windspeed_avg = round(sum(response["hourly"]["windspeed_10m"]) / len(response["hourly"]["windspeed_10m"]), 2)
    rain_avg = round(sum(response["hourly"]["rain"]) / len(response["hourly"]["rain"]), 2)

    # Sunset
    sunset_seconds, ftr = [], [60, 1]
    sunset_res = [val.split("T")[1] for val in response["daily"]["sunset"]]
    for val in sunset_res: 
      sunset_seconds.append(sum([a*b for a,b in zip(ftr, map(int,val.split(':')))]))
    sunset_avg = round(sum(sunset_seconds) / len(sunset_seconds), 2)

    city_data = [temp_avg, humidity_avg, windspeed_avg, rain_avg, sunset_avg]
    weather_data.append(city_data)

weather_data = pd.DataFrame(weather_data)
print(weather_data)

       0      1      2     3        4
0  76.37  62.31  12.15  0.17  1205.38
1  83.25  77.94   8.68  0.03   244.48
2  82.48  37.10   9.61  0.00   503.05
3  47.00  91.04   6.33  0.19   388.19
4  74.95  82.33   5.43  0.04   327.86
5  59.35  48.32  11.39  0.01   607.38


In [8]:
# Aesthetic Data (2 / 4)





In [113]:
# Safety Data (3 / 4)

# Source: FBI
# Reference: https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/docApi
# Reliability: 6

crimes = ["aggravated-assault", "violent-crime", "robbery", "arson", "human-trafficing", "rape-legacy", "homicide", "burglary", "motor-vehicle-theft", "larceny", "rape", "property-crime"]

state_crimes = []
KEY = env['FBI']
for city in cities.keys():
    crime_list = []
    state = cities[city]['state']
    for crime in crimes:
        URL = f"https://api.usa.gov/crime/fbi/cde/estimate/state/{state}/{crime}?from=2020&to=2023&API_KEY={KEY}"
        response = requests.get(url = URL).json()

        if response and response["results"] != "null":
            response = response["results"]
            try: 
                crime_stat = next(iter(response.values()))
            except:
                crime_list.append(-1)
                continue
            crime_stat = next(iter(crime_stat.values()))
            if isinstance(crime_stat, (int, float, complex)) and not isinstance(crime_stat, bool):
                crime_list.append(crime_stat)
            else:
                crime_list.append(-1)
        else:
            crime_list.append(-1)
    state_crimes.append(crime_list)

state_crimes = pd.DataFrame(state_crimes)
print(state_crimes)

      0      1      2     3   4   5    6      7      8       9   10      11
0  240.7  363.8   90.6   5.2  -1  -1  4.2  165.5  101.7  1143.6  -1  1410.7
1  288.7  442.0  113.6  29.9  -1  -1  5.6  369.7  427.6  1341.7  -1  2138.9
2  278.9  425.9   97.4  12.6  -1  -1  9.1  246.4  169.5  1143.5  -1  1559.4
3  302.5  446.5   91.4  10.1  -1  -1  6.6  370.7  287.0  1587.3  -1  2245.0
4  350.2  484.8   83.7  17.2  -1  -1  6.9  330.0  241.7  1656.0  -1  2227.7
5  264.6  389.5   83.9   4.3  -1  -1  7.9  248.6  121.0  1274.5  -1  1644.1


In [11]:
# Infrastructure Data (4 / 4)

# Source: OpenStreetMapsNX
# Reference: https://osmnx.readthedocs.io/en/stable/
# Reliability: 10

transportation = ["walk", "bike", "drive"]

connection_data = []
for city in cities.keys():
  if city == 'New_York':
    city_maps = []
    city_name = f"{city}, {cities[city]['state']}, {cities[city]['country']}"

    for method in transportation:
      if method == 'drive':
        try:
          G = ox.graph_from_place(city_name, network_type=method)
          city_maps.append(nx.adjacency_matrix(G).todense())
        except Exception: pass

    connection_data.append(city_maps)

print(connection_data)

[[array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 2, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])]]
